<a href="https://colab.research.google.com/github/AnuttaraR/DSGP_Group_03/blob/main/New_MediaPipe_Pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#Installing and importing necessary libraries

!pip install mediapipe
!pip install keras

import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pickle
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.preprocessing import LabelEncoder


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Giving list of words
words = ['angry','bank', 'brother','bye','excuse me','father', 'good evening', 'good morning', 'good night', 'happy','hello', 'help','home', 'hospital','how much', 'hungry','love', 'mother', 'police station', 'sad', 'school', 'sister','sorry','thankyou', 'welcome', 'what', 'when', 'where', 'who','why']
len(words)

30

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
dic = {}
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    for word in words:
        dire = os.path.join('/content/drive/MyDrive/MediaPipe_Data/Dataset_MediPipe/src_2', word)
        vids = os.listdir(dire)
        vid_no = 0
        print(word)
        dic[word] = []
        for i,vid in enumerate(vids):
            dic[word].append([])
            vidcap = cv2.VideoCapture(os.path.join('/content/drive/MyDrive/MediaPipe_Data/Dataset_MediPipe/src_2', word, vid))
            success,frame = vidcap.read()
            count = 0
            while success:     
                success,frame = vidcap.read()
                if success:
                    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = hands.process(image)
                    if results.multi_hand_landmarks:
                        # Loop through each hand landmark in the list
                        for hand_landmarks in results.multi_hand_landmarks:
                            # Get the landmarks for the hand
                            landmarks = hand_landmarks.landmark
                            landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                            # Store the landmarks in the dataset array                             
                            dic[word][i].append(landmarks_array.flatten())
                            # Draw the hand landmarks on the image for visualization
                            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
            vid_no +=1
        

angry
bank
brother
bye
excuse me
father
good evening
good morning
good night
happy
hello
help
home
hospital
how much
hungry
love
mother
police station
sad
school
sister
sorry
thankyou
welcome
what
when
where
who
why


In [9]:
# create a binary pickle file 
f = open("/content/drive/MyDrive/MediaPipe_Data/Dataset_PKL_MediaPipe/preprocessed_all.pkl","wb")
pickle.dump(dic,f)
print("Successfuly transfered")
f.close()

Successfuly transfered


In [10]:


with open('/content/drive/MyDrive/MediaPipe_Data/Dataset_PKL_MediaPipe/preprocessed_all.pkl', 'rb') as f:
    data = pickle.load(f)
print("reading is successful")

def stretch(video, size):
    arr = np.array(video)
    n = len(arr)
    x = np.linspace(0, n - 1, n)
    new_x = np.linspace(0, n - 1, size)
    new_arr = np.zeros((size, len(video[0])))
    for i in range(size):
        new_arr[:, i] = np.interp(new_x, x, arr[:, i])
    
    return new_arr



reading is successful


In [11]:
for key in data.keys():
    print(key, len(data[key][1]))

angry 17
bank 111
brother 32
bye 38
excuse me 65
father 31
good evening 56
good morning 65
good night 74
happy 52
hello 34
help 59
home 19
hospital 104
how much 10
hungry 21
love 41
mother 30
police station 118
sad 27
school 75
sister 27
sorry 50
thankyou 34
welcome 35
what 17
when 29
where 9
who 19
why 24


In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

src_2 = {} 
for key in data.keys():
    arr = data[key]
    src_2[key] = []
    for vid in arr:
        if len(vid) !=0 :
            src_2[key].append(stretch(vid, 63))

minn = 200
for arr in src_2.values():
    for ar in arr:
        minn = min(minn, len(ar))
        
print(minn)

src_3 = {}
for key in src_2.keys():
    if len(src_2[key]) <= 63 and len(src_2[key]) >= 10:
        src_3[key] = src_2[key]

for key in src_3.keys():
    print(key, len(src_3[key]))



63
angry 25
bank 25
brother 25
bye 25
excuse me 25
father 25
good evening 25
good morning 25
good night 25
happy 25
hello 25
help 25
home 25
hospital 25
how much 25
hungry 25
love 25
mother 25
police station 25
sad 25
school 25
sister 25
sorry 25
thankyou 25
welcome 25
what 25
when 25
where 25
who 25
why 25


In [19]:
# create a binary pickle file 
f = open("/content/drive/MyDrive/MediaPipe_Data/Dataset_PKL_MediaPipe/framed_all.pkl","wb")
pickle.dump(src_3,f)
print("Successfuly transfered")
f.close()

Successfuly transfered


In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

words = src_2.keys()

label_map = {label:num for num, label in enumerate(words)}

print('label map: ', label_map)


label map:  {'angry': 0, 'bank': 1, 'brother': 2, 'bye': 3, 'excuse me': 4, 'father': 5, 'good evening': 6, 'good morning': 7, 'good night': 8, 'happy': 9, 'hello': 10, 'help': 11, 'home': 12, 'hospital': 13, 'how much': 14, 'hungry': 15, 'love': 16, 'mother': 17, 'police station': 18, 'sad': 19, 'school': 20, 'sister': 21, 'sorry': 22, 'thankyou': 23, 'welcome': 24, 'what': 25, 'when': 26, 'where': 27, 'who': 28, 'why': 29}


In [21]:
print("bye: ", src_3['bye'][0][0][0])

bye:  0.39737629890441895


In [22]:

sequences, labels = [], []
for word in words:
    for i,sequence in enumerate(src_3[word]):
        window = src_3[word][i][:63]
        sequences.append(np.asarray(window).astype('float32'))
        labels.append(label_map[word])

print('sequence shape: ', np.array(sequences).shape)

print("label shape: ", np.array(labels).shape)


sequence shape:  (750, 63, 63)
label shape:  (750,)


In [23]:
X = np.array(sequences)

print("array shape: ", X.shape)

array shape:  (750, 63, 63)


In [24]:
y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print("y_test shape: ", y_test.shape)

y_test shape:  (150, 30)


In [28]:
print("X_train shape: ", X_train.shape)

X_train shape:  (600, 63, 63)


In [26]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [32]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Create the model
model = tf.keras.models.Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(len(words), activation='softmax')
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['categorical_accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2000, callbacks=[tb_callback])

#saving the model
model.save("/content/drive/MyDrive/MediaPipe_Data/Model_MediaPipe/NEW_Tf_mp_model")

Epoch 1/2000
19/19 [==============================] - 2s 24ms/step - loss: 3.4211 - categorical_accuracy: 0.0317 - val_loss: 3.3172 - val_categorical_accuracy: 0.0800
Epoch 2/2000
19/19 [==============================] - 0s 11ms/step - loss: 3.2839 - categorical_accuracy: 0.0783 - val_loss: 3.1994 - val_categorical_accuracy: 0.1267
Epoch 3/2000
19/19 [==============================] - 0s 17ms/step - loss: 3.1407 - categorical_accuracy: 0.1067 - val_loss: 3.0151 - val_categorical_accuracy: 0.1267
Epoch 4/2000
19/19 [==============================] - 0s 21ms/step - loss: 2.9796 - categorical_accuracy: 0.1433 - val_loss: 2.7945 - val_categorical_accuracy: 0.2733
Epoch 5/2000
19/19 [==============================] - 0s 19ms/step - loss: 2.8029 - categorical_accuracy: 0.2117 - val_loss: 2.6391 - val_categorical_accuracy: 0.3133
Epoch 6/2000
19/19 [==============================] - 0s 9ms/step - loss: 2.6102 - categorical_accuracy: 0.2850 - val_loss: 2.4238 - val_categorical_accuracy: 0.4067

No Validation data 

In [41]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Create the model
model_2 = tf.keras.models.Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(len(words), activation='softmax')
])

# Compile the model
model_2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['categorical_accuracy'])

# Train the model
model_2.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

#saving the model
model_2.save("/content/drive/MyDrive/MediaPipe_Data/Model_MediaPipe/NEW_Tf_mp_model_noVal")

Epoch 1/2000
19/19 [==============================] - 1s 7ms/step - loss: 3.3718 - categorical_accuracy: 0.0450
Epoch 2/2000
19/19 [==============================] - 0s 7ms/step - loss: 3.1533 - categorical_accuracy: 0.0917
Epoch 3/2000
19/19 [==============================] - 0s 7ms/step - loss: 2.9161 - categorical_accuracy: 0.1500
Epoch 4/2000
19/19 [==============================] - 0s 6ms/step - loss: 2.6795 - categorical_accuracy: 0.2667
Epoch 5/2000
19/19 [==============================] - 0s 6ms/step - loss: 2.4778 - categorical_accuracy: 0.2800
Epoch 6/2000
19/19 [==============================] - 0s 6ms/step - loss: 2.3113 - categorical_accuracy: 0.3400
Epoch 7/2000
19/19 [==============================] - 0s 7ms/step - loss: 2.1642 - categorical_accuracy: 0.3650
Epoch 8/2000
19/19 [==============================] - 0s 7ms/step - loss: 2.0039 - categorical_accuracy: 0.4450
Epoch 9/2000
19/19 [==============================] - 0s 7ms/step - loss: 1.9150 - categorical_accuracy:

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 61, 32)            6080      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 30, 32)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 960)               0         
                                                                 
 dense_1 (Dense)             (None, 30)                28830     
                                                                 
Total params: 34,910
Trainable params: 34,910
Non-trainable params: 0
_________________________________________________________________


In [42]:
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 61, 32)            6080      
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 30, 32)           0         
 1D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 960)               0         
                                                                 
 dense_3 (Dense)             (None, 30)                28830     
                                                                 
Total params: 34,910
Trainable params: 34,910
Non-trainable params: 0
_________________________________________________________________


In [34]:
res = model.predict(X_test)

print("X_test shape: ", X_test.shape)


5/5 [==============================] - 0s 4ms/step
X_test shape:  (150, 63, 63)


In [43]:
res_2 = model_2.predict(X_test)

print("X_test shape: ", X_test.shape)

5/5 [==============================] - 0s 5ms/step
X_test shape:  (150, 63, 63)


In [44]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

yhat = model.predict(X_test)
yhat_2 = model_2.predict(X_test)


5/5 [==============================] - 0s 4ms/step


In [45]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
yhat_2 = np.argmax(yhat_2, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)
print()
multilabel_confusion_matrix(ytrue, yhat_2)

array([[[145,   1],
        [  1,   3]],

       [[144,   1],
        [  0,   5]],

       [[147,   0],
        [  0,   3]],

       [[143,   0],
        [  1,   6]],

       [[141,   2],
        [  0,   7]],

       [[144,   0],
        [  0,   6]],

       [[146,   2],
        [  1,   1]],

       [[145,   0],
        [  0,   5]],

       [[142,   1],
        [  2,   5]],

       [[144,   0],
        [  0,   6]],

       [[145,   1],
        [  0,   4]],

       [[145,   0],
        [  2,   3]],

       [[147,   0],
        [  0,   3]],

       [[143,   0],
        [  0,   7]],

       [[144,   0],
        [  0,   6]],

       [[145,   0],
        [  0,   5]],

       [[145,   0],
        [  0,   5]],

       [[143,   0],
        [  0,   7]],

       [[148,   0],
        [  1,   1]],

       [[145,   0],
        [  0,   5]],

       [[147,   0],
        [  0,   3]],

       [[144,   0],
        [  0,   6]],

       [[143,   0],
        [  0,   7]],

       [[145,   0],
        [  1, 

In [53]:
print("Accuracy with validation datasets: ", accuracy_score(ytrue, yhat))
print()
print("Improved accuracy without validation datatsets", accuracy_score(ytrue, yhat_2))


Accuracy with validation datasets:  0.9133333333333333

Improved accurqacy without validation datatsets 0.9333333333333333


In [38]:
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5) as hands:
    vidcap = cv2.VideoCapture('/content/drive/MyDrive/MediaPipe_Data/Test_Video_MediaPipe/love_P5_1.mp4')
    success,frame = vidcap.read()
    count = 0
    test = []
    while success:     
        success,frame = vidcap.read()
        if success:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = hand_landmarks.landmark
                    landmarks_array = np.array([[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
                    test.append(landmarks_array.flatten())

nptest = np.array([test[:35]])

print("test shape: ", nptest.shape)


test shape:  (1, 35, 63)


In [54]:
z = model.predict(nptest)

z_2 = model_2.predict(nptest)


InvalidArgumentError: ignored